# IMPORTS

In [4]:
import pandas as pd
import numpy as np 
import pyreadstat as pt

# CONCAT EVERY DATA FILE

In [5]:
'''
desc : from a filename 'fichier' (with an ".sav" extension) returns a DataFrame
return : DataFrame
'''
def lire(fichier):
    df, meta = pt.read_sav(fichier)
    return df

'''
desc : concat multiple dataframe, joining the same columns and adding the ones that are different
dfs : a list of DataFrame
return : the concatenation of dfs
'''
def concat(dfs):
    res = dfs[0]
    for i in range(1, len(dfs)):
        res = res.merge(dfs[i], how="outer")
    return res


In [6]:
path = "../TER_M1_MIASHS/donnees/"
files = ["Base_SA_globale_20182019.sav", "cahier6_final_Prev_20172018.sav", "Cahier6_def_Prev_20162017.sav", "Cahier6_Prev_20152016.sav", "Cahier6_Prev_20142015.sav"]

df_by_year = []
for f in files:
    df = lire(path + f)

    annee = f.split('.')[0].split('_')
    annee = annee[len(annee)-1]
    df["YEAR"] = annee

    df_by_year.append(df)

PyreadstatError: ignored

In [ ]:
raw_data = concat(df_by_year)

# SAVES ALL DATA CONCATENED IN A CSV FILE
raw_data.to_csv(path + 'donnees.csv')
raw_data.head()

,REG,PROV,COM,VIL,MEN,CENQ,NB_REPA,TYP_JR,CONS_CER1,CONS_CER2,...,Type_men,Classe_SDA,S62Q2,S62Q3_1,S62Q3_2,S62Q3_3,S62Q2_c,S62Q4_3_cor1,S62Q2_1,PrimaryLast3
0,1.0,13.0,1.0,1.0,45.0,76.0,3.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,13.0,1.0,1.0,58.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,13.0,1.0,1.0,67.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,13.0,1.0,1.0,75.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,13.0,1.0,1.0,84.0,76.0,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# DATA CLEANSING

Certaines colonnes ont des noms différents et représentant la même métrique, d'autres ne nous intéresse tout simplement pas.
On essaye alors de réduire le nombre de colonnes.

In [ ]:
for col in raw_data.columns:
  print(col)

In [ ]:
col = ["S62Q2","S62Q3_1","S62Q3_2","S62Q3_3","S62Q2_c","S62Q4_3_cor1","S62Q2_1", "YEAR"]


col = ["S63Q3_F","S63Q3_G","S63Q3_H","S63Q3_I","S63Q3_J","S63Q3_K", "S63Q1_F"] + col
print(col)
raw_data[col]

['S63Q3_F', 'S63Q3_G', 'S63Q3_H', 'S63Q3_I', 'S63Q3_J', 'S63Q3_K', 'S63Q1_F', 'S62Q2', 'S62Q3_1', 'S62Q3_2', 'S62Q3_3', 'S62Q2_c', 'S62Q4_3_cor1', 'S62Q2_1', 'YEAR']


,S63Q3_F,S63Q3_G,S63Q3_H,S63Q3_I,S63Q3_J,S63Q3_K,S63Q1_F,S62Q2,S62Q3_1,S62Q3_2,S62Q3_3,S62Q2_c,S62Q4_3_cor1,S62Q2_1,YEAR
0,7000.0,0.0,6000.0,0.0,0.0,0.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
1,18000.0,9000.0,0.0,0.0,0.0,0.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
2,25000.0,63000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
3,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20182019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,4.0,5.0,3.0,10000.0,3.0,20142015
25981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,4.0,5.0,3.0,15000.0,3.0,20142015
25982,2000.0,30000.0,15000.0,45000.0,0.0,0.0,0.0,3.0,3.0,4.0,2.0,3.0,60000.0,3.0,20142015
25983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,4.0,5.0,3.0,10000.0,3.0,20142015


NameError: ignored